### 如果有社保， socialSecurityList，利用保障缺口精算法计算总保额时，减去医疗保险预估值的一半。

In [1]:
def get_testament(start,end,x,life_cost,support_old,education,house,car,age_little_children,
                                age_little_elder, socialSecurityList):

    elder_need_year_num = 76 - age_little_elder
    children_need_year_num = 22 - age_little_children

    if len(socialSecurityList) > 0:
        return (end - start) * (life_cost ) + house*2 + car + support_old*(elder_need_year_num) +  education*(children_need_year_num) - (50+5*x)*0.6*0.5
    else:
        return  (end - start) * (life_cost ) + house*2 + car + support_old*(elder_need_year_num) +  education*(children_need_year_num)

### 保额配比用到的新增加的函数

In [2]:
def bmi(weight, height):
    sg = height / 100.
    BMI = round(weight / (sg * sg))
    return BMI

def quzheng(x):
    return round(x/10) * 10

### 保额分配思路

In [3]:
def baoe_distribution(data, person_risk, min_child, age, sex, work_type, year_money, bad_habit, BMI, content_list, socialSecurity, flag=1):
    '''

    :param data:        bao_dic，由不同计算方式计算得到的user保额估算
    :param person_risk: 个人收入占家庭总收入的系数【1，10】
    :param min_child:   最小孩子的年龄
    :param age:         user_age
    :param sex:         user_sex
    :param work_type:   user_worktype,高危险等级[1,7]。
    :param year_money:  user_年收入
    :param bad_habit:   是否喝酒抽烟坏习惯
    :param BMI:         BMI体质系数
    :param conten_list: 用户标语，用来推荐理由
    :param socialSecurity ：用户已有社保list
    :param flag:        flag 为用户所选择的推荐方式。
                        flag = 1 : 高性价比推荐方案，固定搭配（重疾+定寿+百万医疗+意外）
                        flag = 2 : 高保障方案，固定搭配（百万医疗+重疾+两全+意外）
    :return:
        info：           保险产品推荐理由
        type_and_money： 保额险种配比
    '''
    all_money = data['保障缺口精算法保额']
    live_money_base = 50 + year_money * 5  # 保障出意外，生存： 重疾 + 医疗
    dead_money_base = 100 + year_money * 5  # 保障出意外，死掉： 人寿 + 意外
    stt = get_xiugang(bad_habit)  # 不良习惯
    socialSecurity = socialSecurity

    if flag == 1:
        type_and_money = {'重疾': 0, '定寿': 0, '意外': 0, '医疗': 0}

        info = '您选择了高性价比方案,根据缺口精算法为您计算保额需求为：{}\n'.format(all_money)
        info += '\n下面给出保险推荐理由：\n'

        bad_habit_flag = 0
        BMI_flag = 0
        work_type_flag = 0

        if len(bad_habit) >= 0:
            bad_habit_flag = 1
            info += '您有不良健康习惯：' + stt + ',需要预防不良习惯引起的病症。'
            live_money_base += 10 * len(bad_habit)
        else:
            info += '您无不良健康习惯。'
            live_money_base -= 20

        if BMI < 18.5:
            BMI_flag = 1
            info += '您的体质系数BMI={},低于正常体重，患病风险有所提高。'.format(BMI)
            live_money_base += int(5 * (18.5 - BMI))
        elif 18.5 <= BMI < 25:
            info += '您的体质系数BMI={},属于正常体重。'.format(BMI)
            live_money_base -= 20
        elif BMI >= 25:
            BMI_flag = 1
            info += '您的体质系数BMI={},超出正常体重，患病风险有所提高。'.format(BMI)
            live_money_base += int(5 * (BMI - 25))

        if work_type >= 4:
            work_type_flag = 1
            info += '您的工作属于体力劳动较多或者高危职业，危险等级较高（{}/7）, 需要一定的医疗保障需要提前预防意外的发生。\n'.format(work_type)
            live_money_base += (work_type - 3) * 20
        else:
            info += '您的工作危险等级为{}/7，相对较安全。\n'.format(work_type)
            live_money_base += (work_type - 3) * 20

        if live_money_base >= 200:
            live_money_base = 200
        elif live_money_base <= 100:
            live_money_base = 100

        # 身体不太健康的，抽烟喝酒的重疾多一点，职业高风险的医疗多一点。
        if work_type_flag:
            zhongji_money = quzheng(int(live_money_base * 0.3))
            if len(socialSecurity) > 0:
                yiliao_money = quzheng(int(live_money_base * 0.7)) * 0.5
            else:
                yiliao_money = quzheng(int(live_money_base * 0.7))
        else:
            zhongji_money = quzheng(int(live_money_base * 0.4))
            if len(socialSecurity) > 0:
                yiliao_money = quzheng(int(live_money_base * 0.7)) * 0.5
            else:
                yiliao_money = quzheng(int(live_money_base * 0.7))
            yiliao_money = quzheng(int(yiliao_money))

        if sex == 1 or person_risk >= 7:
            info += '考虑到以上因素，作为家庭支柱,'
        else:
            info += '考虑到以上因素，'

        info += '若不幸发生重大疾病，需要大量治疗费用，推荐您考虑重疾保险与医疗保险搭配，其保额分别为：重疾{}万元、医疗{}万元' \
                '（需要自行承担的是百万医疗险中1万免赔额以内的医疗开支）。\n'.format(zhongji_money, yiliao_money)

        # 工作危险等级低的可以降低意外险的部分，投入到寿险里。
        yiwai_money = year_money * 5
        dingshou_money = 100
        if work_type < 4:
            cz_money = (4 - work_type) * 20
            yiwai_money -= cz_money
            dingshou_money += cz_money
        else:
            yiwai_money += (work_type - 3) * 20

        pre_money = yiwai_money + yiliao_money + dingshou_money +zhongji_money
        if all_money - pre_money > 0:
            dingshou_money += all_money - pre_money



        if work_type >= 4:
            info += '您的职业危险系数较高，'
        else:
            info += ''
        info += '若不幸发生意外，推荐您考虑意外保险与定寿保险搭配，其保额分别为：意外{}万元、定寿{}万元'.format(yiwai_money, dingshou_money)


        type_and_money['定寿'] = quzheng(dingshou_money)
        type_and_money['重疾'] = quzheng(zhongji_money)
        type_and_money['意外'] = quzheng(yiwai_money)
        type_and_money['医疗'] = quzheng(yiliao_money)
        print('计划估算各险种保额：{}'.format(type_and_money))

        # 如果超出保额10%， 各部分保额按照比列减小。
        if round((pre_money-all_money)/all_money, 2) > 0.1:
            type_and_money['定寿'] *= all_money / pre_money
            type_and_money['重疾'] *= all_money / pre_money
            type_and_money['意外'] *= all_money / pre_money
            type_and_money['医疗'] *= all_money / pre_money
        # 如果低于总保额10%， 各部分保额按照比列减小。
        elif round((all_money - pre_money)/all_money, 2) > 0.1:
            type_and_money['定寿'] *= all_money / pre_money
            type_and_money['重疾'] *= all_money / pre_money
            type_and_money['意外'] *= all_money / pre_money
            type_and_money['医疗'] *= all_money / pre_money

        type_and_money['定寿'] = quzheng(type_and_money['定寿'])
        type_and_money['重疾'] = quzheng(type_and_money['重疾'])
        type_and_money['意外'] = quzheng(type_and_money['意外'])
        type_and_money['医疗'] = quzheng(type_and_money['医疗'])

        return info, type_and_money


    if flag == 2:
        type_and_money = {'重疾': 0, '两全': 0, '意外': 0, '医疗': 0}

        info = '您选择了高保障方案,根据缺口精算法为您计算保额需求为：{}\n'.format(all_money)
        info += '\n下面给出保险推荐理由：\n'

        bad_habit_flag = 0
        BMI_flag = 0
        work_type_flag = 0

        if len(bad_habit) >= 0:
            bad_habit_flag = 1
            info += '您有不良健康习惯：' + stt + ',需要预防不良习惯引起的病症。'
            live_money_base += 10 * len(bad_habit)
        else:
            info += '您无不良健康习惯。'
            live_money_base -= 20

        if BMI < 18.5:
            BMI_flag = 1
            info += '您的体质系数BMI={},低于正常体重，患病风险有所提高。'.format(BMI)
            live_money_base += int(5 * (18.5 - BMI))
        elif 18.5 <= BMI < 25:
            info += '您的体质系数BMI={},属于正常体重。'.format(BMI)
            live_money_base -= 20
        elif BMI >= 25:
            BMI_flag = 1
            info += '您的体质系数BMI={},超出正常体重，患病风险有所提高。'.format(BMI)
            live_money_base += int(5 * (BMI - 25))

        if work_type >= 4:
            work_type_flag = 1
            info += '您的工作属于体力劳动较多或者高危职业，危险等级较高（{}/7）, 需要一定的医疗保障需要提前预防意外的发生。\n'.format(work_type)
            live_money_base += (work_type - 3) * 20
        else:
            info += '您的工作危险等级为{}/7，相对较安全。\n'.format(work_type)
            live_money_base += (work_type - 3) * 20

        if live_money_base >= 300:
            live_money_base = 300
        elif live_money_base <= 100:
            live_money_base = 100

        # 身体不太健康的，抽烟喝酒的重疾多一点，职业高风险的医疗多一点。
        if work_type_flag:
            zhongji_money = quzheng(int(live_money_base * 0.3))
            if len(socialSecurity) > 0:
                yiliao_money = quzheng(int(live_money_base * 0.7)) * 0.5
            else:
                yiliao_money = quzheng(int(live_money_base * 0.7))
        else:
            zhongji_money = quzheng(int(live_money_base * 0.4))
            if len(socialSecurity) > 0:
                yiliao_money = quzheng(int(live_money_base * 0.7)) * 0.5
            else:
                yiliao_money = quzheng(int(live_money_base * 0.7))
            yiliao_money = quzheng(int(yiliao_money))

        if sex == 1 or person_risk >= 7:
            info += '考虑到以上因素，作为家庭支柱,'
        else:
            info += '考虑到以上因素，'

        info += '若不幸发生重大疾病，需要大量治疗费用，推荐您考虑重疾保险与医疗保险搭配，其保额分别为：重疾{}万元、医疗{}万元' \
                '（需要自行承担的是百万医疗险中1万免赔额以内的医疗开支）。\n'.format(zhongji_money, yiliao_money)

        # 工作危险等级低的可以降低意外险的部分，投入到寿险里。
        yiwai_money = year_money * 5
        liangquan_money = 100
        if work_type < 4:
            cz_money = (4 - work_type) * 20
            yiwai_money -= cz_money
            liangquan_money += cz_money
        else:
            yiwai_money += (work_type - 3) * 20

        pre_money = yiwai_money + yiliao_money + liangquan_money +zhongji_money
        if all_money - pre_money > 0:
            liangquan_money += all_money - pre_money



        if work_type >= 4:
            info += '您的职业危险系数较高，'
        else:
            info += ''
        info += '若不幸发生意外，推荐您考虑意外保险与两全保险搭配，其保额分别为：意外{}万元、定寿{}万元'.format(yiwai_money, liangquan_money)


        type_and_money['两全'] = quzheng(liangquan_money)
        type_and_money['重疾'] = quzheng(zhongji_money)
        type_and_money['意外'] = quzheng(yiwai_money)
        type_and_money['医疗'] = quzheng(yiliao_money)
        print('计划估算各险种保额：{}'.format(type_and_money))

        # 如果超出保额10%， 各部分保额按照比列减小。
        if round((pre_money-all_money)/all_money, 2) > 0.1:
            type_and_money['两全'] *= all_money / pre_money
            type_and_money['重疾'] *= all_money / pre_money
            type_and_money['意外'] *= all_money / pre_money
            type_and_money['医疗'] *= all_money / pre_money
        # 如果低于总保额10%， 各部分保额按照比列减小。
        elif round((all_money - pre_money)/all_money, 2) > 0.1:
            type_and_money['两全'] *= all_money / pre_money
            type_and_money['重疾'] *= all_money / pre_money
            type_and_money['意外'] *= all_money / pre_money
            type_and_money['医疗'] *= all_money / pre_money

        type_and_money['两全'] = quzheng(type_and_money['两全'])
        type_and_money['重疾'] = quzheng(type_and_money['重疾'])
        type_and_money['意外'] = quzheng(type_and_money['意外'])
        type_and_money['医疗'] = quzheng(type_and_money['医疗'])

        return info, type_and_money